In [11]:
from ipyleaflet import Map, Marker, MarkerCluster, AwesomeIcon, Popup, basemaps, basemap_to_tiles
from ipywidgets import HTML

import pandas as pd

In [12]:
def get_popup_html_message_for_hospital(name, count_sick, count_healthy, count_dead):
    text ='<h2>' + name + '</h2>' + '<p><b>Заболени:</b>' + str(count_sick) + "</p>" + "<p><b>Излечени:</b>" + str(count_healthy) + "</p>" + "<p><b>Мртви:</b>" + str(count_dead) + "</p>"
    message = HTML()
    message.value = text
    return message
    
def get_popup_for_hospital(message):
    popup = Popup(
        child=message,
        close_button=True,
        auto_close=False,
        close_on_escape_key=False
    )
    return popup

In [17]:
city_data = pd.read_csv("../podatoci/mkcities.csv")

city_data

,id,city_cyrilic,admin_name_cyrilic,city_latin,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population
0,1807600615,Скопје,Скопје,Skopje,Skopje,Skopje,42.0000,21.4335,Macedonia,MK,MKD,Skopje,False,494087.0
1,1807020286,Тетово,Тетово,Tetovo,Tetovo,Tetovo,42.0092,20.9701,Macedonia,MK,MKD,Tetovo,False,119132.0
2,1807488567,Битола,Битола,Bitola,Bitola,Bitola,41.0391,21.3395,Macedonia,MK,MKD,Bitola,False,86528.0
3,1807691189,Сопиште,Сопиште,Sopište,Sopište,Sopiste,41.9547,21.4275,Macedonia,MK,MKD,Sopište,False,NaN
4,1807895120,Кичево,Кичево,Kičevo,Kičevo,Kicevo,41.5127,20.9589,Macedonia,MK,MKD,Kičevo,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,1807953626,Белишта,Дебарца,Belčišta,Belčišta,Belcista,41.3028,20.8303,Macedonia,MK,MKD,Debarca,False,NaN
67,1807782793,Старо Нагоричане,Старо Нагоричане,Staro Nagoričane,Staro Nagoričane,Staro Nagoricane,42.1981,21.8286,Macedonia,MK,MKD,Staro Nagoričane,False,NaN
68,1807831162,Македонски Брод,Македонски Брод,Makedonski Brod,Makedonski Brod,Makedonski Brod,41.5136,21.2153,Macedonia,MK,MKD,Makedonski Brod,False,NaN
69,1807936735,Новаци,Новаци,Novaci,Novaci,Novaci,41.0420,21.4587,Macedonia,MK,MKD,Novaci,False,NaN


In [14]:


center=(41.6086, 21.7453)

m = Map(
    layers=(basemap_to_tiles(basemaps.OpenStreetMap.Mapnik, "2017-04-08"), ),
    center=center,
    zoom=8
)

hospital_icon = AwesomeIcon(
    name='h-square',
    marker_color='red',
    icon_color='white',
    spin=False
)

In [15]:

hospital_list = list()


for ind in hospital_data.index:
    location = (hospital_data["Longitude"][ind], hospital_data["Latitude"][ind])
    marker = Marker(location=location, draggable=False, icon=hospital_icon)
    marker.popup = get_popup_for_hospital(get_popup_html_message_for_hospital("Klinika", 4, 5, 1))
    hospital_list.append(marker)

    
    
hospitals = MarkerCluster(markers=tuple(hospital_list))
m.add_layer(hospitals)

m


NameError: name 'hospital_data' is not defined